In [361]:
import glob
import pandas as pd
import numpy as np

import os
import re

In [396]:
attack = "crownibp"

In [397]:
results_folder = rf"results\{attack}"

In [398]:
algorithms = ["baseline","noise","fgsm","pgd","crownibp"]

In [399]:
# CI , IBS , NegLL
metric = "NegLL"

In [400]:
ascending = False if metric=="CI" else True

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{metric}.xlsx")
    excels.extend(glob.glob(glob_search))

In [401]:
df = pd.DataFrame()
N_datasets = len(excels)
percentage_change = []
print(excels)
for excel_i in excels:

    temp_df = pd.read_excel(excel_i)
    folder_name,dataset_name = excel_i.split("\\")[-3:-1]
    temp_df.columns = ["eps"] + temp_df.columns[1:].to_list()
    col_name = (dataset_name,re.sub("results_","",folder_name))
    df[col_name] = temp_df[f"Robust {metric}"] #temp_df["Non Robust CI"].round(3).astype(str) + " / " + temp_df["Robust CI"].round(3).astype(str)

    col_name = (dataset_name,"baseline")
    df[col_name] = temp_df[f"Non Robust {metric}"]

df.index = temp_df.eps

['results\\crownibp\\results_crownibp\\Aids2\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\dataDIVAT1\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\Dialysis\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\divorce\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\flchain\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\Framingham\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\LeukSurv\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\Pbc3\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\prostate\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\retinopathy\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\stagec\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\TRACE\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\vlbw\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\zinc\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\Aids2\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\dataDIVAT1\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\Dialysis\\Neg

In [402]:
micolumns = pd.MultiIndex.from_tuples(df.columns)
dfmi = (
    pd.DataFrame(
        df.values,
        index=df.index,
        columns=micolumns,
    )
    .sort_index()
    .sort_index(axis=1)
)

dfmi.sort_values(by="eps",ascending=False,inplace=True)
dfmi = dfmi.reindex(columns=algorithms, level=1)


In [403]:
dfmi[dfmi==''] = np.NaN
dfmi = dfmi.astype(float)
dfmi

Aids2                                                         \
           baseline         noise          fgsm           pgd     crownibp   
eps                                                                          
2.00  867218.625000  1.664442e+09  5.247136e+07  4.170300e+07  1013.211182   
1.00   11408.425781  3.843492e+05  6.768741e+04  5.493030e+04   529.279114   
0.80    4595.645508  6.678125e+04  1.776805e+04  1.489971e+04   522.198975   
0.70    2868.485596  2.779730e+04  8.894230e+03  7.577893e+03   521.521118   
0.60    1798.874268  1.135636e+04  4.351412e+03  3.794776e+03   521.171753   
0.50    1159.978516  4.424974e+03  2.142487e+03  1.915283e+03   520.945190   
0.10     515.993896  5.196252e+02  5.187775e+02  5.186812e+02   521.655640   
0.07     516.228821  5.188468e+02  5.188582e+02  5.188345e+02   521.751038   
0.05     516.784729  5.191172e+02  5.191102e+02  5.190440e+02   521.819092   
0.00     518.514893  5.208565e+02  5.195742e+02  5.194845e+02   521.989746   

          Dialysis                                                          \
          baseline         noise          fgsm           pgd      crownibp   
eps                                                                          
2.00  4.559872e+16           NaN           NaN  2.248880e+33  29887.589844   
1.00  9.240066e+08  6.920299e+34  7.336558e+19  3.604828e+16   2584.852295   
0.80  3.504494e+07  2.610555e+27  8.823269e+15  2.004266e+13   1959.104614   
0.70  7.014061e+06  5.569553e+23  9.986558e+13  4.851810e+11   1776.846558   
0.60  1.445596e+06  1.300917e+20  1.149362e+12  1.224906e+10   1658.694092   
0.50  3.095362e+05  3.559136e+16  1.365481e+10  3.267395e+08   1591.970093   
0.10  1.812542e+03  5.482956e+03  1.955184e+03  1.672969e+03   1523.746338   
0.07  1.644429e+03  2.364513e+03  1.608519e+03  1.546026e+03   1523.031616   
0.05  1.579474e+03  1.844645e+03  1.542314e+03  1.521537e+03   1522.824341   
0.00  1.516727e+03  1.567583e+03  1.521137e+03  1.514367e+03   1523.203857   

      ...          vlbw                                            \
      ...      baseline         noise          fgsm           pgd   
eps   ...                                                           
2.00  ...  9.966138e+17           NaN  1.607044e+30  7.045944e+27   
1.00  ...  6.365167e+08  5.456794e+17  7.582620e+13  5.521332e+12   
0.80  ...  1.405074e+07  4.117141e+13  1.024871e+11  1.066556e+10   
0.70  ...  2.233256e+06  4.700205e+11  3.847962e+09  4.988304e+08   
0.60  ...  3.588179e+05  6.820365e+09  1.494893e+08  2.427380e+07   
0.50  ...  5.837540e+04  1.209359e+08  5.977126e+06  1.207330e+06   
0.10  ...  1.165087e+02  1.811381e+02  1.480437e+02  9.331113e+01   
0.07  ...  8.887656e+01  1.041134e+02  1.060349e+02  6.751047e+01   
0.05  ...  7.527724e+01  7.563555e+01  8.892874e+01  5.563836e+01   
0.00  ...  5.109546e+01  4.125416e+01  6.073362e+01  3.795073e+01   

                            zinc                                            \
          crownibp      baseline         noise          fgsm           pgd   
eps                                                                          
2.00  9.197520e+12  1.725447e+13  8.805155e+15  5.331321e+13  3.805171e+13   
1.00  1.242647e+05  2.621423e+06  2.929443e+07  2.503320e+06  2.546512e+06   
0.80  1.132517e+04  1.326696e+05  7.761540e+05  9.493627e+04  8.271138e+04   
0.70  4.563683e+03  3.129429e+04  1.320493e+05  2.105466e+04  1.874853e+04   
0.60  1.872497e+03  7.434188e+03  2.336807e+04  4.885672e+03  4.430068e+03   
0.50  7.776690e+02  1.868499e+03  4.454716e+03  1.243473e+03  1.133289e+03   
0.10  1.037632e+02  8.256099e+01  8.171708e+01  7.661854e+01  7.659334e+01   
0.07  9.203265e+01  7.886884e+01  7.833759e+01  7.559594e+01  7.570814e+01   
0.05  8.503030e+01  7.711427e+01  7.689701e+01  7.531229e+01  7.548814e+01   
0.00  6.996465e+01  7.463243e+01  7.533926e+01  7.565608e+01  7.597625e+01   

                     
           crownibp  
eps                  
2.00 

In [404]:
# dfmi.T.groupby(axis=0,level=0).rank(axis=0,na_option='bottom',method="average",ascending=ascending)

In [405]:
best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)
best_per_dataset_rank

C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\996602392.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)
C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\996602392.py:1: FutureWarning: The 'axis' keyword in DataFrameGroupBy.rank is deprecated and will be removed in a future version. Call without passing 'axis' instead.
  best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)


Aids2                          Dialysis                           ...  \
     baseline noise fgsm  pgd crownibp baseline noise fgsm  pgd crownibp  ...   
eps                                                                       ...   
2.00      2.0   5.0  4.0  3.0      1.0      2.0   4.5  4.5  3.0      1.0  ...   
1.00      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.80      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.70      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.60      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.50      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.10      1.0   4.0  3.0  2.0      5.0      3.0   5.0  4.0  2.0      1.0  ...   
0.07      1.0   3.0  4.0  2.0      5.0      4.0   5.0  3.0  2.0      1.0  ...   
0.05      1.0   4.0  3.0  2.0      5.0      4.0   5.0  3.0  1.0      2.0  ...   
0.00      1.0   4.0  3.0  2.0      5.0      2.0   5.0  3.0  1.0      4.0  ...   

         vlbw                              zinc                           
     baseline noise fgsm  pgd crownibp baseline noise fgsm  pgd crownibp  
eps                                                                       
2.00      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  
1.00      2.0   5.0  4.0  3.0      1.0      4.0   5.0  2.0  3.0      1.0  
0.80      2.0   5.0  4.0  3.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.70      2.0   5.0  4.0  3.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.60      2.0   5.0  4.0  3.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.50      2.0   5.0  4.0  3.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.10      3.0   5.0  4.0  1.0      2.0      5.0   4.0  2.0  1.0      3.0  
0.07      2.0   4.0  5.0  1.0      3.0      5.0   4.0  1.0  2.0      3.0  
0.05      2.0   3.0  5.0  1.0      4.0      4.0   3.0  1.0  2.0      5.0  
0.00      3.0   2.0  4.0  1.0      5.0      1.0   2.0  3.0  4.0      5.0  

[10 rows x 70 columns]

In [406]:
best_per_dataset_avg_rank = best_per_dataset_rank.stack(level=1).mean(1).unstack(1).sort_values(by="eps",ascending=False)
best_per_dataset_avg_rank

,baseline,noise,fgsm,pgd,crownibp
eps,,,,,
2.00,2.857143,4.321429,3.750000,3.071429,1.000000
1.00,3.000000,4.500000,3.500000,3.000000,1.000000
0.80,2.928571,4.571429,3.571429,2.928571,1.000000
0.70,3.000000,4.571429,3.500000,2.928571,1.000000
0.60,2.928571,4.714286,3.500000,2.857143,1.000000
0.50,2.928571,4.714286,3.500000,2.857143,1.000000
0.10,3.357143,4.571429,3.071429,2.214286,1.785714
0.07,3.428571,4.142857,3.142857,2.357143,1.928571
0.05,3.285714,4.071429,3.000000,2.357143,2.285714


In [407]:
if not ascending:
    best_per_dataset = dfmi.groupby(level=0,axis=1).idxmax(1)
else:
    best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
best_per_dataset.applymap(lambda x: x[1])

C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\1722459016.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\1722459016.py:4: FutureWarning: DataFrameGroupBy.idxmin with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\1722459016.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1])


,Aids2,Dialysis,Framingham,LeukSurv,Pbc3,TRACE,dataDIVAT1,divorce,flchain,prostate,retinopathy,stagec,vlbw,zinc
eps,,,,,,,,,,,,,,
2.00,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
1.00,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.80,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.70,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.60,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.50,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.10,baseline,crownibp,crownibp,crownibp,baseline,crownibp,crownibp,fgsm,crownibp,crownibp,crownibp,crownibp,pgd,pgd
0.07,baseline,crownibp,crownibp,crownibp,noise,crownibp,crownibp,fgsm,crownibp,crownibp,crownibp,crownibp,pgd,fgsm
0.05,baseline,pgd,crownibp,crownibp,noise,crownibp,baseline,fgsm,crownibp,crownibp,crownibp,crownibp,pgd,fgsm


In [408]:
if metric != "NegLL":
     # aggregate all the CI files
    os.listdir(results_folder)
    excels = []
    for folder in os.listdir(results_folder):
        glob_search = os.path.join(results_folder,folder,f"{metric}_all.xlsx")
        excels.extend(glob.glob(glob_search))
        
    df = pd.DataFrame()
    N_datasets = len(excels)
    percentage_change = []
    for excel_i in excels:
    
        temp_df = pd.read_excel(excel_i)
        folder_name = excel_i.split("\\")[-2]
        temp_df.columns = ["eps"] + temp_df.columns[1:].to_list()
        col_name = re.sub("results_","",folder_name)
        df[col_name] = temp_df["%"] #temp_df["Non Robust CI"].round(3).astype(str) + " / " + temp_df["Robust CI"].round(3).astype(str)
    
    df.index = temp_df.eps
    df = df.reindex(columns=algorithms[1:], level=1)

In [409]:
df

,"(Aids2, crownibp)","(Aids2, baseline)","(dataDIVAT1, crownibp)","(dataDIVAT1, baseline)","(Dialysis, crownibp)","(Dialysis, baseline)","(divorce, crownibp)","(divorce, baseline)","(flchain, crownibp)","(flchain, baseline)",...,"(flchain, pgd)","(Framingham, pgd)","(LeukSurv, pgd)","(Pbc3, pgd)","(prostate, pgd)","(retinopathy, pgd)","(stagec, pgd)","(TRACE, pgd)","(vlbw, pgd)","(zinc, pgd)"
eps,,,,,,,,,,,,,,,,,,,,,
2.00,1013.211182,867218.625000,20130.279297,3.917180e+06,29887.589844,4.559872e+16,1093.994751,1.627015e+07,1.400884e+12,NaN,...,inf,1.258766e+14,2.998791e+30,5.015566e+11,1.691516e+26,6.762288e+06,3.632862e+10,8.507489e+18,7.045944e+27,3.805171e+13
1.00,529.279114,11408.425781,918.089050,1.433132e+04,2584.852295,9.240066e+08,1051.740601,2.363391e+04,6.307257e+03,3.350622e+23,...,1.304035e+16,2.014873e+07,1.284540e+15,4.558995e+05,8.837224e+12,7.303170e+03,1.538292e+05,4.530066e+08,5.521332e+12,2.546512e+06
0.80,522.198975,4595.645508,821.152588,4.946042e+03,1959.104614,3.504494e+07,1048.827271,6.934156e+03,1.425336e+03,2.564323e+18,...,4.929555e+11,9.573399e+05,1.031830e+12,3.244131e+04,2.175150e+10,2.043456e+03,1.469834e+04,5.348799e+06,1.066556e+10,8.271138e+04
0.70,521.521118,2868.485596,800.327820,3.008757e+03,1776.846558,7.014061e+06,1047.909302,4.025990e+03,1.194910e+03,7.372112e+15,...,3.470649e+09,2.129752e+05,2.670271e+10,9.031110e+03,1.136398e+09,1.108950e+03,4.627388e+03,6.115734e+05,4.988304e+08,1.874853e+04
0.60,521.171753,1798.874268,785.855286,1.914498e+03,1658.694092,1.445596e+06,1047.239380,2.521232e+03,1.115522e+03,2.230515e+13,...,4.597551e+07,4.876955e+04,7.957938e+08,2.552643e+03,5.942480e+07,6.254221e+02,1.474163e+03,7.626407e+04,2.427380e+07,4.430068e+03
0.50,520.945190,1159.978516,774.627319,1.311128e+03,1591.970093,3.095362e+05,1046.801880,1.788535e+03,1.089056e+03,6.824825e+10,...,1.331019e+06,1.211749e+04,2.300900e+07,7.240753e+02,3.266468e+06,3.810739e+02,4.837622e+02,1.071335e+04,1.207330e+06,1.133289e+03
0.10,521.655640,515.993896,751.842834,7.562355e+02,1523.746338,1.812542e+03,1045.666992,1.056980e+03,1.069681e+03,2.033814e+03,...,1.102356e+03,1.499732e+03,4.654940e+02,3.124055e+01,6.400184e+02,1.682910e+02,3.934553e+01,5.403820e+02,9.331113e+01,7.659334e+01
0.07,521.751038,516.228821,751.301270,7.522886e+02,1523.031616,1.644429e+03,1045.636963,1.050426e+03,1.069362e+03,1.443374e+03,...,1.084690e+03,1.495175e+03,3.881605e+02,3.113774e+01,5.438411e+02,1.674806e+02,3.899010e+01,5.340093e+02,6.751047e+01,7.570814e+01
0.05,521.819092,516.784729,751.017822,7.505762e+02,1522.824341,1.579474e+03,1045.620850,1.047734e+03,1.069202e+03,1.250709e+03,...,1.078688e+03,1.493774e+03,3.583383e+02,3.125700e+01,5.001221e+02,1.672798e+02,3.891127e+01,5.309935e+02,5.563836e+01,7.548814e+01


In [410]:
excel_name = os.path.join(results_folder,f"{metric}_all.xlsx")
with pd.ExcelWriter(excel_name) as writer:  
    dfmi.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
    best_per_dataset_rank.to_excel(writer,sheet_name="rank")
    best_per_dataset_avg_rank.to_excel(writer,sheet_name="average_rank")
    best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")
    df.to_excel(writer,sheet_name="%")

C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\229516633.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfmi.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\229516633.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")


In [411]:
best_per_dataset_avg_rank.index = ["{:.2f}".format(float(x)) for x in np.round(best_per_dataset_avg_rank.index.tolist(),2)]
best_per_dataset_avg_rank.index.name = r"$\epsilon$"
print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))

\begin{tabular}{llllll}
\toprule
 & baseline & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  &  \\
\midrule
2.00 & 2.86 & 4.32 & 3.75 & 3.07 & 1.0 \\
1.00 & 3.0 & 4.5 & 3.5 & 3.0 & 1.0 \\
0.80 & 2.93 & 4.57 & 3.57 & 2.93 & 1.0 \\
0.70 & 3.0 & 4.57 & 3.5 & 2.93 & 1.0 \\
0.60 & 2.93 & 4.71 & 3.5 & 2.86 & 1.0 \\
0.50 & 2.93 & 4.71 & 3.5 & 2.86 & 1.0 \\
0.10 & 3.36 & 4.57 & 3.07 & 2.21 & 1.79 \\
0.07 & 3.43 & 4.14 & 3.14 & 2.36 & 1.93 \\
0.05 & 3.29 & 4.07 & 3.0 & 2.36 & 2.29 \\
0.00 & 2.71 & 3.79 & 3.36 & 2.64 & 2.5 \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\3581541270.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))


In [412]:
if metric == "NegLL":
    dfmi.index = ["{:.2f}".format(float(x)) for x in np.round(dfmi.index.tolist(),2)]
    dfmi.index.name = r"$\epsilon$"
    print(dfmi.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))
else:
    dfmi.index = ["{:.2f}".format(float(x)) for x in np.round(dfmi.index.tolist(),2)]
    dfmi.index.name = r"$\epsilon$"
    print(dfmi.applymap(lambda x: np.round(x,3)).applymap(str).to_latex(index=True,multicolumn_format="c"))

C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\718377934.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(dfmi.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))


\begin{tabular}{lllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll}
\toprule
 & \multicolumn{5}{c}{Aids2} & \multicolumn{5}{c}{Dialysis} & \multicolumn{5}{c}{Framingham} & \multicolumn{5}{c}{LeukSurv} & \multicolumn{5}{c}{Pbc3} & \multicolumn{5}{c}{TRACE} & \multicolumn{5}{c}{dataDIVAT1} & \multicolumn{5}{c}{divorce} & \multicolumn{5}{c}{flchain} & \multicolumn{5}{c}{prostate} & \multicolumn{5}{c}{retinopathy} & \multicolumn{5}{c}{stagec} & \multicolumn{5}{c}{vlbw} & \multicolumn{5}{c}{zinc} \\
 & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & bas

In [413]:
dataset_names = np.array(list(map(np.array,dfmi.columns)))[:,0]

In [414]:
dataset_names

array(['Aids2', 'Aids2', 'Aids2', 'Aids2', 'Aids2', 'Dialysis',
       'Dialysis', 'Dialysis', 'Dialysis', 'Dialysis', 'Framingham',
       'Framingham', 'Framingham', 'Framingham', 'Framingham', 'LeukSurv',
       'LeukSurv', 'LeukSurv', 'LeukSurv', 'LeukSurv', 'Pbc3', 'Pbc3',
       'Pbc3', 'Pbc3', 'Pbc3', 'TRACE', 'TRACE', 'TRACE', 'TRACE',
       'TRACE', 'dataDIVAT1', 'dataDIVAT1', 'dataDIVAT1', 'dataDIVAT1',
       'dataDIVAT1', 'divorce', 'divorce', 'divorce', 'divorce',
       'divorce', 'flchain', 'flchain', 'flchain', 'flchain', 'flchain',
       'prostate', 'prostate', 'prostate', 'prostate', 'prostate',
       'retinopathy', 'retinopathy', 'retinopathy', 'retinopathy',
       'retinopathy', 'stagec', 'stagec', 'stagec', 'stagec', 'stagec',
       'vlbw', 'vlbw', 'vlbw', 'vlbw', 'vlbw', 'zinc', 'zinc', 'zinc',
       'zinc', 'zinc'], dtype='<U11')

In [415]:
dfmi.T

$\epsilon$              2.00          1.00           0.80           0.70  \
Aids2 baseline  8.672186e+05  1.140843e+04    4595.645508    2868.485596   
      noise     1.664442e+09  3.843492e+05   66781.250000   27797.304688   
      fgsm      5.247136e+07  6.768741e+04   17768.054688    8894.230469   
      pgd       4.170300e+07  5.493030e+04   14899.712891    7577.892578   
      crownibp  1.013211e+03  5.292791e+02     522.198975     521.521118   
...                      ...           ...            ...            ...   
zinc  baseline  1.725447e+13  2.621423e+06  132669.562500   31294.289062   
      noise     8.805155e+15  2.929443e+07  776154.000000  132049.328125   
      fgsm      5.331321e+13  2.503320e+06   94936.273438   21054.656250   
      pgd       3.805171e+13  2.546512e+06   82711.382812   18748.529297   
      crownibp  5.558544e+05  1.312633e+03     471.883026     297.965393   

$\epsilon$              0.60         0.50        0.10        0.07        0.05  \
Aids2 baseline   1798.874268  1159.978516  515.993896  516.228821  516.784729   
      noise     11356.355469  4424.974121  519.625244  518.846802  519.117188   
      fgsm       4351.411621  2142.486816  518.777527  518.858154  519.110168   
      pgd        3794.776367  1915.282837  518.681152  518.834534  519.044006   
      crownibp    521.171753   520.945190  521.655640  521.751038  521.819092   
...                      ...          ...         ...         ...         ...   
zinc  baseline   7434.187988  1868.499146   82.560989   78.868835   77.114273   
      noise     23368.070312  4454.716309   81.717079   78.337585   76.897011   
      fgsm       4885.671875  1243.473022   76.618538   75.595940   75.312286   
      pgd        4430.067871  1133.288818   76.593338   75.708138   75.488136   
      crownibp    198.103073   141.571533   78.744469   78.210060   77.983910   

$\epsilon$            0.00  
Aids2 baseline  518.514893  
      noise     520.856506  
      fgsm      519.574158  
      pgd       519.484497  
      crownibp  521.989746  
...                    ...  
zinc  baseline   74.632431  
      noise      75.339256  
      fgsm       75.656082  
      pgd        75.976250  
      crownibp   77.801697  

[70 rows x 10 columns]

In [416]:
dfmi.T.swaplevel(i=1,j=0,axis=0).groupby(level=0).mean().reindex(algorithms)

$\epsilon$,2.00,1.00,0.80,0.70,0.60,0.50,0.10,0.07,0.05,0.00
baseline,7.017447e+18,2.393302e+22,1.831659e+17,5.265794e+14,1.593225e+12,4.874909e+09,674.099620,610.177975,587.368147,565.116555
noise,3.121070e+17,2.051513e+34,2.406121e+26,4.312432e+22,9.540865e+18,2.561187e+15,5828.892042,3609.117438,2747.372702,1559.203304
fgsm,1.749824e+33,1.675503e+19,6.881433e+14,7.301683e+12,8.310990e+10,9.903465e+08,687.436978,629.695854,611.144597,590.841025
pgd,inf,3.599122e+15,1.542873e+12,3.692914e+10,9.411149e+08,2.540309e+07,617.893978,592.316392,583.708293,574.122638
crownibp,7.570577e+11,1.236275e+04,1.865039e+03,1.100089e+03,7.876175e+02,6.555492e+02,566.986664,565.653699,564.924817,563.609678


In [417]:
dfmi.T.groupby(level=0).mean()

$\epsilon$,2.00,1.00,0.80,0.70,0.60,0.50,0.10,0.07,0.05,0.00
Aids2,3.518970e+08,1.037809e+05,2.091337e+04,9.531887e+03,4.364518e+03,2.032733e+03,518.946692,518.903870,519.175037,520.083960
Dialysis,7.496268e+32,1.384060e+34,5.221111e+26,1.113911e+23,2.601835e+19,7.118274e+15,2489.479663,1737.303735,1602.158667,1528.603564
Framingham,2.992924e+13,6.981889e+06,3.959240e+05,9.887332e+04,2.614093e+04,7.874911e+03,1514.517603,1503.848022,1499.189966,1494.200049
LeukSurv,5.249695e+33,1.092426e+23,1.596936e+18,5.030407e+15,1.705891e+13,3.762908e+10,850.221378,600.417255,503.944519,381.054794
Pbc3,1.304059e+14,2.142671e+06,8.571270e+04,1.795947e+04,4.091805e+03,9.919316e+02,31.032850,30.498335,30.412217,30.867180
TRACE,7.258175e+24,4.131167e+10,9.877680e+07,5.872072e+06,4.211124e+05,3.963104e+04,567.719482,550.032092,541.926184,530.355640
dataDIVAT1,4.117338e+24,3.707124e+10,7.247999e+07,3.915635e+06,2.489291e+05,2.210616e+04,762.359668,758.479895,757.047253,756.030579
divorce,5.947479e+06,9.609457e+03,3.330157e+03,2.199126e+03,1.616900e+03,1.321667e+03,1048.556299,1046.554980,1045.759619,1045.406128
flchain,inf,4.360176e+34,1.516026e+26,9.357019e+21,6.960605e+17,5.302358e+13,1463.193799,1214.995288,1142.072437,1083.568970
prostate,1.249567e+29,1.400782e+20,7.490189e+15,5.694511e+13,4.568502e+11,4.053896e+09,13788.197491,8482.245215,6263.714008,3083.179352


In [418]:
# pd.set_option('display.max_columns', None)
# print(dfmi)

In [419]:
print(best_per_dataset.applymap(lambda x: x[1]).to_latex(index=False))

\begin{tabular}{llllllllllllll}
\toprule
Aids2 & Dialysis & Framingham & LeukSurv & Pbc3 & TRACE & dataDIVAT1 & divorce & flchain & prostate & retinopathy & stagec & vlbw & zinc \\
\midrule
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & cr

C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\3574660769.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset.applymap(lambda x: x[1]).to_latex(index=False))


In [420]:
df.index = ["{:.2f}".format(float(x)) for x in np.round(df.index.tolist(),2)]
df.index.name = r"$\epsilon$"
print(df.applymap(lambda x: str(np.round(x,3))).to_latex(index=True))

\begin{tabular}{lllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll}
\toprule
 & ('Aids2', 'crownibp') & ('Aids2', 'baseline') & ('dataDIVAT1', 'crownibp') & ('dataDIVAT1', 'baseline') & ('Dialysis', 'crownibp') & ('Dialysis', 'baseline') & ('divorce', 'crownibp') & ('divorce', 'baseline') & ('flchain', 'crownibp') & ('flchain', 'baseline') & ('Framingham', 'crownibp') & ('Framingham', 'baseline') & ('LeukSurv', 'crownibp') & ('LeukSurv', 'baseline') & ('Pbc3', 'crownibp') & ('Pbc3', 'baseline') & ('prostate', 'crownibp') & ('prostate', 'baseline') & ('retinopathy', 'crownibp') & ('retinopathy', 'baseline') & ('stagec', 'crownibp') & ('stagec', 'baseline') & ('TRACE', 'crownibp') & ('TRACE', 'baseline') & ('vlbw', 'crownibp') & ('vlbw', 'baseline') & ('zinc', 'crownibp') & ('zinc', 'baseline') & ('Aids2', 'fgsm') & ('dataDIVAT1', 'fgsm') & ('Dialysis', 'fgsm') & ('divorce', 'fgsm') & ('flchain', 'fgsm') & ('Framingham', 'fgsm') & ('LeukSurv', 'fgsm') & ('Pbc3', 'fgs

C:\Users\lpott\AppData\Local\Temp\ipykernel_23016\3740848601.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df.applymap(lambda x: str(np.round(x,3))).to_latex(index=True))


## PDF PICTURES

In [246]:
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import pandas as pd
import re

In [49]:
attack = "crownibp"

In [50]:
results_folder = rf"results\{attack}"

In [51]:
algorithms = ["baseline","noise","fgsm","pgd","crownibp"]

In [52]:
# CI , IBS , NegLL
img_name = "population_curves_attacked_test"

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{img_name}.xlsx")
    excels.extend(glob.glob(glob_search))

In [53]:
metadata = 'eps'

In [54]:
n_rows = int(len(excels) / (len(algorithms)-1))
n_cols = int(len(algorithms) - 1)

In [ ]:
excels = np.sort(excels)
files = pd.DataFrame(excels.reshape(-1,len(algorithms)-1,order="F"),columns=["crownibp","fgsm","noise","pgd"]).reindex(["noise","fgsm","pgd","crownibp"],axis=1).values

In [ ]:
df_population = pd.read_excel(excels[0])

In [ ]:
df_population

In [ ]:
save_folder = results_folder = os.path.join(fr"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results\{attack}","perturb_curves.pdf")

In [ ]:
print(save_folder)

In [ ]:
save_folder = results_folder = os.path.join(r"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results",attack,"perturb_curves.pdf")

fig,axes = plt.subplots(n_rows,n_cols+1,figsize=(30,40),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(n_rows):
    for j in range(n_cols):
        print(i,j)
        fileij = files[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        df_population= df_population.iloc[:,3:]

        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in robust_df.columns]
        baseline_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in baseline_df.columns]
        
        base1 = axes[i][j+1].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j+1].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        base3 = axes[i][j+1].plot(t,base_models.iloc[:,2],linewidth=3,c="k")

  
        robust1 = axes[i][j+1].plot(t,robust_df,'--',linewidth=3)
  
        if j == 0:
            base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            base3 = axes[i][j].plot(t,base_models.iloc[:,2],linewidth=3,c="k")
            baseline1 = axes[i][j].plot(t,baseline_df,'--',linewidth=3)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)

        axes[i][j+1].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = base_models.columns.tolist() + robust_df.columns.tolist()
fig.legend([base1, base2,base3,robust1], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.06)) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

In [ ]:
files[2:4,:]

In [ ]:
save_folder = results_folder = os.path.join(r"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results",attack,"perturb_curves_subset.pdf")
file_subset = files[2:4,:]

fig,axes = plt.subplots(2,n_cols+1,figsize=(30,12),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(2):
    for j in range(n_cols):
        print(i,j)
        fileij = file_subset[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        if dataset not in ["TRACE","Framingham"]:
            continue
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        df_population= df_population.iloc[:,3:]

        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in robust_df.columns]
        baseline_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in baseline_df.columns]
        
        base1 = axes[i][j+1].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j+1].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        base3 = axes[i][j+1].plot(t,base_models.iloc[:,2],linewidth=3,c="k")

  
        robust1 = axes[i][j+1].plot(t,robust_df,'--',linewidth=3)
  
        if j == 0:
            base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            base3 = axes[i][j].plot(t,base_models.iloc[:,2],linewidth=3,c="k")
            baseline1 = axes[i][j].plot(t,baseline_df,'--',linewidth=3)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)

        axes[i][j+1].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = base_models.columns.tolist() + robust_df.columns.tolist()
fig.legend([base1, base2,base3,robust1], labels=labels, 
           loc="upper center",ncols=6,fontsize=30,bbox_to_anchor=(.5,1.15)) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

### DIST PLOTS

In [ ]:
# CI , IBS , NegLL
results_folder = fr"results\{attack}"
img_name = "curve_distributions_test"

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{img_name}.xlsx")
    excels.extend(glob.glob(glob_search))

In [ ]:
excels = np.sort(excels)
files = pd.DataFrame(excels.reshape(-1,len(algorithms)-1,order="F"),columns=["crownibp","fgsm","noise","pgd"]).reindex(["noise","fgsm","pgd","crownibp"],axis=1).values

In [ ]:
save_folder = results_folder = os.path.join(r"results",attack,"dist_curves.pdf")

In [ ]:
import seaborn as sns

In [ ]:
fig,axes = plt.subplots(n_rows,n_cols+1,figsize=(30,40),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(n_rows):
    for j in range(n_cols):
        print(i,j)
        fileij = files[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = [col.split("_")[1] for col in robust_df.columns]
        baseline_df.columns =[col.split("_")[1] for col in baseline_df.columns]
        
        mu = sns.lineplot(x=t, y=robust_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j+1],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=robust_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=robust_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)

        line = q05.get_lines()
        axes[i][j+1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        axes[i][j+1].set_xlabel("t",fontsize=20)

        if j == 0:
            mu = sns.lineplot(x=t, y=baseline_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
            q95 = sns.lineplot(x=t, y=baseline_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
            q05 = sns.lineplot(x=t, y=baseline_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)
            line = q05.get_lines()
            axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
            axes[i][j].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.03)) 

plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

In [ ]:
save_folder = results_folder = os.path.join(r"results",attack,"dist_curves_subset.pdf")
file_subset = files[2:4,:]

fig,axes = plt.subplots(2,n_cols+1,figsize=(30,12),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(2):
    for j in range(n_cols):
        print(i,j)
        fileij = file_subset[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = [col.split("_")[1] for col in robust_df.columns]
        baseline_df.columns =[col.split("_")[1] for col in baseline_df.columns]
        
        mu = sns.lineplot(x=t, y=robust_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j+1],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=robust_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=robust_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        axes[i][j+1].set_xlabel("t",fontsize=20)

        line = q05.get_lines()
        axes[i][j+1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        if j == 0:
            mu = sns.lineplot(x=t, y=baseline_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
            q95 = sns.lineplot(x=t, y=baseline_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
            q05 = sns.lineplot(x=t, y=baseline_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)
            line = q05.get_lines()
            axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
            axes[i][j].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.12)) 

plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()